In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
from sksurv.metrics import concordance_index_censored, integrated_brier_score
from sksurv.util import Surv
from sksurv.ensemble import GradientBoostingSurvivalAnalysis
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from joblib import dump

import warnings

warnings.filterwarnings("ignore")

seed = 32

**Цель** - обучить алгоритм анализа на выживаемость данных ухода клиентов из данных, аугментировать данные и разработать платформу для предиктивного анализа.

# Загрузка данных

In [2]:
df = pd.read_csv('data/data.csv')
df['Churn'] = df['Churn'].apply(lambda x: 1 if x == 'Yes' else 0)
df

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,0
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.50,0
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,1
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,0
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7038,6840-RESVB,Male,0,Yes,Yes,24,Yes,Yes,DSL,Yes,...,Yes,Yes,Yes,Yes,One year,Yes,Mailed check,84.80,1990.50,0
7039,2234-XADUH,Female,0,Yes,Yes,72,Yes,Yes,Fiber optic,No,...,Yes,No,Yes,Yes,One year,Yes,Credit card (automatic),103.20,7362.90,0
7040,4801-JZAZL,Female,0,Yes,Yes,11,No,No phone service,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.60,346.45,0
7041,8361-LTMKD,Male,1,Yes,No,4,Yes,Yes,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Mailed check,74.40,306.60,1


## Удаление лишних столбцов

In [3]:
delete_cols = ['customerID']
df = df.loc[:, ~df.columns.isin(delete_cols)]

## Разделение на типы данных

In [4]:
cat_cols = [
    "gender",
    "SeniorCitizen",
    "Partner",
    "Dependents",
    "PhoneService",
    "MultipleLines",
    "InternetService",
    "OnlineSecurity",
    "OnlineBackup",
    "DeviceProtection",
    "TechSupport",
    "StreamingTV",
    "StreamingMovies",
    "Contract",
    "PaperlessBilling",
    "PaymentMethod"
]
num_cols = [
    "MonthlyCharges",
    "TotalCharges"
]
censor_label = "Churn"
value_label = "tenure"

# Визуализация

## Категориальные признаки

In [11]:
cat_counts = df[cat_cols[0]].value_counts()
px.bar(x=cat_counts.index, y=cat_counts.values, title=cat_cols[0])

In [12]:
cat_counts = df[cat_cols[1]].value_counts()
px.bar(x=cat_counts.index, y=cat_counts.values, title=cat_cols[1])

In [13]:
cat_counts = df[cat_cols[2]].value_counts()
px.bar(x=cat_counts.index, y=cat_counts.values, title=cat_cols[2])

In [14]:
cat_counts = df[cat_cols[3]].value_counts()
px.bar(x=cat_counts.index, y=cat_counts.values, title=cat_cols[3])

In [15]:
cat_counts = df[cat_cols[4]].value_counts()
px.bar(x=cat_counts.index, y=cat_counts.values, title=cat_cols[4])

In [16]:
cat_counts = df[cat_cols[5]].value_counts()
px.bar(x=cat_counts.index, y=cat_counts.values, title=cat_cols[5])

In [17]:
cat_counts = df[cat_cols[6]].value_counts()
px.bar(x=cat_counts.index, y=cat_counts.values, title=cat_cols[6])

In [18]:
cat_counts = df[cat_cols[7]].value_counts()
px.bar(x=cat_counts.index, y=cat_counts.values, title=cat_cols[7])

In [19]:
cat_counts = df[cat_cols[8]].value_counts()
px.bar(x=cat_counts.index, y=cat_counts.values, title=cat_cols[8])

In [20]:
cat_counts = df[cat_cols[9]].value_counts()
px.bar(x=cat_counts.index, y=cat_counts.values, title=cat_cols[9])

In [21]:
cat_counts = df[cat_cols[10]].value_counts()
px.bar(x=cat_counts.index, y=cat_counts.values, title=cat_cols[10])

In [22]:
cat_counts = df[cat_cols[11]].value_counts()
px.bar(x=cat_counts.index, y=cat_counts.values, title=cat_cols[11])

In [23]:
cat_counts = df[cat_cols[12]].value_counts()
px.bar(x=cat_counts.index, y=cat_counts.values, title=cat_cols[12])

In [24]:
cat_counts = df[cat_cols[13]].value_counts()
px.bar(x=cat_counts.index, y=cat_counts.values, title=cat_cols[13])

In [25]:
cat_counts = df[cat_cols[14]].value_counts()
px.bar(x=cat_counts.index, y=cat_counts.values, title=cat_cols[14])

In [26]:
cat_counts = df[cat_cols[15]].value_counts()
px.bar(x=cat_counts.index, y=cat_counts.values, title=cat_cols[15])

## Числовые признаки

In [28]:
px.box(x=df[num_cols[0]], title=num_cols[0])

In [29]:
px.box(x=df[num_cols[1]], title=num_cols[1])

## Целевые метки

In [35]:
cat_counts = df[censor_label].value_counts()
px.bar(x=cat_counts.index, y=cat_counts.values, title=censor_label)

In [36]:
px.box(x=df[value_label], title=value_label)

# Преобразование данных

In [30]:
feature_transformer = ColumnTransformer([
    ('num_cols', StandardScaler(), num_cols),
    ('cat_cols', OneHotEncoder(sparse_output=False, handle_unknown="infrequent_if_exist"), cat_cols)
], n_jobs=-1).fit(df)
feature_transformer

ColumnTransformer(n_jobs=-1,
                  transformers=[('num_cols', StandardScaler(),
                                 ['MonthlyCharges', 'TotalCharges']),
                                ('cat_cols',
                                 OneHotEncoder(handle_unknown='infrequent_if_exist',
                                               sparse_output=False),
                                 ['gender', 'SeniorCitizen', 'Partner',
                                  'Dependents', 'PhoneService', 'MultipleLines',
                                  'InternetService', 'OnlineSecurity',
                                  'OnlineBackup', 'DeviceProtection',
                                  'TechSupport', 'StreamingTV',
                                  'StreamingMovies', 'Contract',
                                  'PaperlessBilling', 'PaymentMethod'])])

In [91]:
dump(feature_transformer, 'api/models/feature_transformer.pkl')

['api/models/feature_transformer.pkl']

In [7]:
X = feature_transformer.transform(df)
y = df[[censor_label, value_label]].to_numpy()

# Обучение модели

In [8]:
from time import time


def fit_score(model):
    # Разделение данных с сохранением стратификации по событию
    X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=seed, stratify=y[:, 0])
    
    # Преобразование целевой переменной для CatBoost (время, событие)
    y_train_cb = np.column_stack([y_train[:, 1], y_train[:, 0]])
    y_test_cb = np.column_stack([y_test[:, 1], y_test[:, 0]])
    
    start_time = time()
    # Обучение модели
    model.fit(X_train, y_train_cb)
    print(f'Fitted for {(time() - start_time):.2f} s')
    
    # Предсказание модели
    y_pred = model.predict(X_test)
    
    # Извлечение событий и времени из тестовой выборки
    event_test = y_test_cb[:, 1]
    time_test = y_test_cb[:, 0]
    
    # Расчет Concordance index
    cindex = concordance_index_censored(event_test.astype(dtype=np.bool), time_test, y_pred)[0]
    print(f"Concordance index: {cindex:.5f}")

In [ ]:
from catboost import CatBoostRegressor

catboost_model = CatBoostRegressor(
    n_estimators=2000,
    learning_rate=0.75,
    random_seed=seed,
    loss_function='SurvivalAft:dist=Logistic;scale=8.5',
    bootstrap_type='MVS',
    depth=7,
    boosting_type='Plain',
    metric_period=50
)

fit_score(catboost_model)

0:	learn: 0.3890367	total: 1.94ms	remaining: 3.88s
50:	learn: 0.2442151	total: 105ms	remaining: 4s
100:	learn: 0.2460017	total: 209ms	remaining: 3.92s
150:	learn: 0.2440034	total: 310ms	remaining: 3.8s
200:	learn: 0.2431743	total: 413ms	remaining: 3.69s
250:	learn: 0.2434929	total: 516ms	remaining: 3.59s
300:	learn: 0.2427592	total: 622ms	remaining: 3.51s
350:	learn: 0.2425202	total: 725ms	remaining: 3.4s
400:	learn: 0.2425212	total: 829ms	remaining: 3.31s
450:	learn: 0.2427129	total: 932ms	remaining: 3.2s
500:	learn: 0.2431385	total: 1.04s	remaining: 3.1s
550:	learn: 0.2446282	total: 1.14s	remaining: 3s
600:	learn: 0.2432544	total: 1.25s	remaining: 2.9s
650:	learn: 0.2431080	total: 1.35s	remaining: 2.81s
700:	learn: 0.2436771	total: 1.46s	remaining: 2.7s
750:	learn: 0.2443753	total: 1.56s	remaining: 2.59s
800:	learn: 0.2478265	total: 1.66s	remaining: 2.49s
850:	learn: 0.2491020	total: 1.77s	remaining: 2.38s
900:	learn: 0.2526030	total: 1.87s	remaining: 2.28s
950:	learn: 0.2500472	tota

In [9]:
model = GradientBoostingSurvivalAnalysis(n_estimators=250, learning_rate=0.4).fit(X, Surv.from_arrays(y[:, 0], y[:, 1], name_event='churn', name_time='tenure'))

In [10]:
dump(model, 'api/models/survival_analysis.pkl')

['api/models/survival_analysis.pkl']

# Аугментация данных

Для аугментации данных создадим вариационный автокодировщик, чтобы воссоздать скрытые зависимости в данных.

In [11]:
import torch
from torch.utils.data import TensorDataset, DataLoader

device = ('cuda' if torch.cuda.is_available() else
          'mps' if torch.mps.is_available() else
          'cpu')

tensor_dataset = TensorDataset(torch.as_tensor(X, dtype=torch.float32, device=device))
loader = DataLoader(tensor_dataset, batch_size=32)

In [ ]:
from typing import Literal

import torch
from torch import nn


class ELBOLoss(nn.Module):
    def __init__(self,
                 reduce: Literal['mean', 'sum'] = 'mean'):
        super(ELBOLoss, self).__init__()
        self.reduce = reduce
        self.reconstruction_loss = nn.MSELoss(reduce=self.reduce)
        self.kl_loss = KullbackLeiblerLoss(reduce=self.reduce)

    def forward(self,
                X_reconstructed: torch.Tensor,
                X_origin: torch.Tensor,
                mu: torch.Tensor,
                log_var: torch.Tensor) -> tuple[torch.Tensor,
                                                torch.Tensor,
                                                torch.Tensor]:
        recon_loss = self.reconstruction_loss(X_reconstructed, X_origin)
        kl_loss = self.kl_loss(log_var, mu)
        loss = recon_loss + kl_loss
        return (loss, recon_loss, kl_loss)


class KullbackLeiblerLoss(nn.Module):
    def __init__(self,
                 reduce: Literal['mean', 'sum'] = 'mean'):
        super(KullbackLeiblerLoss, self).__init__()
        self.reduce = reduce

    def forward(self,
                log_var: torch.Tensor,
                mu: torch.Tensor) -> torch.Tensor:
        losses = -0.5 * torch.sum(1 + log_var - mu ** 2 - log_var.exp(), dim=1)
        return (torch.mean(losses, dim=0) if self.reduce == 'mean'
                else torch.sum(losses, dim=0))



class VariationalAutoencoder(nn.Module):
    def __init__(self,
                 input_shape: int,
                 latent_dim: int,
                 device: str = 'auto'):
        super(VariationalAutoencoder, self).__init__()

        self.activation = nn.SELU()
        if device == 'auto':
            self.device = torch.device(
                'cuda' if torch.cuda.is_available() else
                'mps' if torch.backends.mps.is_available() else
                'cpu'
            )
        else:
            self.device = torch.device(device)
        self.latent_space = latent_dim

        self.encoder = nn.Sequential(
            nn.Linear(input_shape, 256),
            nn.Linear(256, 128),
            self.activation,
            nn.Dropout(0.2),
            nn.Linear(128, 64),
            self.activation,
            nn.Linear(64, latent_dim),
            self.activation
        ).to(self.device)

        self.latent_space_mu = nn.Linear(latent_dim,
                                         latent_dim).to(self.device)
        self.latent_space_var = nn.Linear(latent_dim,
                                          latent_dim).to(self.device)

        self.decoder = nn.Sequential(
            nn.Linear(latent_dim, 64),
            self.activation,
            nn.Linear(64, 128),
            self.activation,
            nn.Dropout(0.2),
            nn.Linear(128, 256),
            self.activation,
            nn.Linear(256, input_shape)
        ).to(self.device)

    def encode(self, X: torch.Tensor) -> tuple[torch.Tensor,
                                               torch.Tensor]:
        encoded = self.encoder(X)
        mu = self.latent_space_mu(encoded)
        log_var = self.latent_space_var(encoded)
        return (mu, log_var)

    def reparametize(self, mu, log_var):
        std = torch.exp(0.5 * log_var)
        noise = torch.randn_like(std).to(self.device)

        z = mu + noise * std
        return z

    def decode(self, z: torch.Tensor) -> torch.Tensor:
        theta = nn.functional.softmax(z, dim=1)
        decoded = self.decoder(theta)
        return decoded

    def forward(self, X: torch.Tensor) -> tuple[torch.Tensor,
                                                torch.Tensor,
                                                torch.Tensor]:
        mu, log_var = self.encode(X)
        z = self.reparametize(mu, log_var)
        X_reconst = self.decode(z)
        return (X_reconst, mu, log_var)

    def generate(self, z: torch.Tensor) -> torch.Tensor:
        return self.decode(z).detach()

    def generate_from_random(self, count: int) -> np.ndarray:
        z = torch.normal(std=1, mean=0,
                         size=(count, self.latent_space),
                         dtype=torch.float32)
        generated = self.generate(z)
        generated[:, 2:] = torch.round(generated[:, 2:])
        return generated.cpu().numpy()


augment_model = VariationalAutoencoder(X.shape[1], 48).to(device)
loss_fn = ELBOLoss()
optimizer = torch.optim.Adagrad(augment_model.parameters(), lr=2e-3)
epochs = 25
augment_model

VariationalAutoencoder(
  (activation): SELU()
  (encoder): Sequential(
    (0): Linear(in_features=45, out_features=256, bias=True)
    (1): Linear(in_features=256, out_features=128, bias=True)
    (2): SELU()
    (3): Dropout(p=0.2, inplace=False)
    (4): Linear(in_features=128, out_features=64, bias=True)
    (5): SELU()
    (6): Linear(in_features=64, out_features=48, bias=True)
    (7): SELU()
  )
  (latent_space_mu): Linear(in_features=48, out_features=48, bias=True)
  (latent_space_var): Linear(in_features=48, out_features=48, bias=True)
  (decoder): Sequential(
    (0): Linear(in_features=48, out_features=64, bias=True)
    (1): SELU()
    (2): Linear(in_features=64, out_features=128, bias=True)
    (3): SELU()
    (4): Dropout(p=0.2, inplace=False)
    (5): Linear(in_features=128, out_features=256, bias=True)
    (6): SELU()
    (7): Linear(in_features=256, out_features=45, bias=True)
  )
)

In [87]:
from tqdm import tqdm

epoch_progress_bar = tqdm(range(1, epochs + 1))
for epoch in epoch_progress_bar:
    running_loss = 0.0
    for features in loader:
        optimizer.zero_grad()
        reconstruct_features, mu, log_var = augment_model(features[0])
        loss, recon_loss, kl_loss = loss_fn(reconstruct_features, features[0], mu, log_var)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    epoch_progress_bar.set_description(f'Epoch {epoch}: loss {running_loss / len(loader):.6f}')

Epoch 25: loss 0.239241: 100%|██████████| 25/25 [00:28<00:00,  1.15s/it]


In [90]:
torch.save(augment_model.cpu().state_dict(), 'augment_model.pth')

In [89]:
generated_data = augment_model.generate_from_random(20)
generated_df = pd.DataFrame(data=np.hstack([feature_transformer.transformers_[0][1].inverse_transform(generated_data[:, :2]), feature_transformer.transformers_[1][1].inverse_transform(generated_data[:, 2:])]), columns=num_cols + cat_cols)
generated_df

,MonthlyCharges,TotalCharges,gender,SeniorCitizen,Partner,Dependents,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod
0,64.596336,2295.601318,Female,0,No,No,Yes,None,None,No,None,None,None,None,None,Month-to-month,Yes,None
1,64.45446,2301.265625,Female,0,No,No,Yes,None,None,No,None,None,None,None,None,Month-to-month,Yes,None
2,64.819054,2307.403076,Female,0,No,No,Yes,None,None,No,None,None,None,None,None,Month-to-month,Yes,None
3,64.584579,2298.312256,Female,0,No,No,Yes,None,None,No,None,None,No,None,None,Month-to-month,Yes,None
4,64.421516,2309.997803,Female,0,No,No,Yes,None,None,No,None,None,No,None,None,Month-to-month,Yes,None
5,64.758179,2310.447021,Female,0,No,No,Yes,None,None,No,None,None,No,None,None,Month-to-month,Yes,None
6,64.83609,2304.512695,Female,0,No,No,Yes,None,None,No,None,No,No,None,None,Month-to-month,Yes,None
7,64.730408,2307.188477,Female,0,No,No,Yes,None,None,No,None,None,No,None,None,Month-to-month,Yes,None
8,65.125923,2330.363037,Male,0,No,No,Yes,None,None,No,None,None,None,None,None,Month-to-month,Yes,None
9,64.596458,2305.950684,Female,0,No,No,Yes,None,None,No,None,None,No,None,None,Month-to-month,Yes,None
